In [2]:
from RLtools.data_processing.stylusprofiler.surfaceprofile import SurfaceProfile
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
import os
sclen = 'Scan Length (um)'
prof = 'Profile (um)'

trace= [f for f in Path(os.getcwd(),'RLTools', 'data_processing/stylusprofiler').rglob('*.txt')]

t=SurfaceProfile(trace[0], 'First Trace')

In [3]:
import numpy as np
#https://stackoverflow.com/a/4495197/7611883
def contiguous_regions(condition):
    """Finds contiguous True regions of the boolean array "condition". Returns
    a 2D array where the first column is the start index of the region and the
    second column is the end index."""

    # Find the indicies of changes in "condition"
    d = np.diff(condition)
    idx, = d.nonzero() 

    # We need to start things after the change in "condition". Therefore, 
    # we'll shift the index by 1 to the right.
    idx += 1

    if condition[0]:
        # If the start of condition is True prepend a 0
        idx = np.r_[0, idx]

    if condition[-1]:
        # If the end of condition is True, append the length of the array
        idx = np.r_[idx, condition.size] # Edit

    # Reshape the result into two columns
    idx.shape = (-1,2)
    return idx

def tulipsplit(df):
    left = df[df[sclen].between(400,6200)].copy(deep = True)
    #left['Name'] = df.name + name1

    right = df[df[sclen].between(6201, 10001)].copy(deep=True)
    #right['Name'] = df.name + name2

    return left, right


In [76]:
import pandas as pd
from scipy.signal import find_peaks, peak_widths
from numpy.polynomial.polynomial import Polynomial as Poly
class CircleStructure(pd.DataFrame):
    
    """docstring for CircleStructure."""
    def __init__(self, name, data):
        super(CircleStructure, self).__init__(data)
        self.name = name
        self.split1 = None 
        self.split2 = None
        self.peaks = {}
    
    

    def add_trace(self, f, **kwargs): 
        return f.add_traces(go.Scatter(x=self[sclen],
                                       y=self[prof],
                                       name = self.name, 
                                       **kwargs))


    def add_child_traces(self,f, **kwargs):
        if isinstance(self.split1, CircleStructure):
            
            self.split1.add_trace(f, **kwargs)
            self.split1.add_child_traces(f, **kwargs)
        
        if isinstance(self.split2, CircleStructure):
            self.split2.add_trace(f, **kwargs)
            self.split2.add_child_traces(f, **kwargs)
            

    def split(self, name1, name2, split_function=None): 
        #split this data based upon an arbitrary split function
        #children are instances of this same class
        
        split1, split2 = split_function(self)
        self.split1 = CircleStructure(name1, split1)
        self.split2 = CircleStructure(name2, split2)

     
    def df_find_peaks(self, h=0.5, d=150, f=None):
        
        ind, ph = find_peaks(self[prof].values, height = h, distance=d)
        print(f'Found {len(ind)} peaks')

        keys = ['widths', 'heights','L','R']
        values = peak_widths(self[prof], ind, rel_height=0.99)
        
        #self.peaks = dict(indices = ind, heights = ph)
        self.peaks= self.iloc[ind]


        for key, value in zip(keys,values):
            self.peaks[key] = value 
        
        #TODO - GET THE X RESOLUTION PROPERLY
        self.peaks['widths'] *= t.x_resolution

        

        if f:
            f.add_trace(go.Scatter(x=self.peaks[sclen], 
                                   y=self.peaks[prof], 
                                    mode='markers', 
                                    marker_symbol='x',
                                    name = t.name + ' peak detection'))
            for ix,row in self.peaks.iterrows():
                
                rescale_x = [(t.x_resolution*row[key]) + self[sclen].min() for key in ['L','R']]
                f.add_trace(go.Scatter(x=rescale_x, y=[row['heights']]*2))


            return f

    def highest_peak(self):
        return self.peaks.query(f'`Profile (um)` == `Profile (um)`.max()')

    
    def fit_peaks(self,f):
        
        fit=Poly.fit(self.peaks[sclen],
                     self.peaks[prof],
                     2)
        print(f'Fitted 2nd order to measured peaks:\n {fit}')


        if f:
            x=np.arange(0,10000)
            y=fit(x)
            mask = y>0
            f.add_trace(go.Scatter(x=x[mask], y=y[mask]))
            return f
    

    

def find_breaks(df, threshold = 0.1, figure = None):
    """
    Find the break in a circle set to allow fitting on either side
    """
    
    y=df[prof].values
 
    true_regions = []
    for start, stop in contiguous_regions(y<threshold):
        segment = y[start:stop]
        seglength = stop-start
        true_regions.append([start,stop,seglength])

    #find the longest region of y<threshold
    longest_ind = np.array(true_regions)[:,2].argmax()
    longest = true_regions[longest_ind]

    #plot the longest region
    longest_break = df.iloc[longest[0]:longest[1]]
    x=longest_break[sclen].values
    
    

    #find the centre point
    mid = int(longest[0] + np.diff(longest[0:2])/2)
    mid = df.iloc[mid]


    
    print(f'Found a breakpoint at {mid[sclen]}')

    if figure:
        #https://plotly.com/python/horizontal-vertical-shapes/
        
        figure.add_vrect(x[0],x[-1], 
                        line_width=0, 
                        fillcolor=trace.line.color, 
                        opacity = 0.2, 
                        annotation_text='discontinuity')

                #plot this point
        figure.add_vline(mid[sclen])

    #split1
    s1 = df[df[sclen].between(0, mid[sclen])]
    s2 = df[df[sclen].between(mid[sclen], 10000)]

    return s1, s2
    


j=CircleStructure('Whole Trace', t.data)
#foo = go.Figure()



ipykernel_launcher:12: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access



In [75]:
j=CircleStructure('Whole Trace', t.data)
j.split('Large Circles','Small Circles',tulipsplit)

for side in [j.split1, j.split2]:
    side.split(f'{side.name}: Left of split',f'{side.name}: Right of split',find_breaks)
j.add_child_traces(foo, fill='tozeroy')

j.split1.split2.df_find_peaks(f=foo)
j.split1.split2.fit_peaks(foo)
j.split1.split2.highest_peak()

Found a breakpoint at 2522.0
Found a breakpoint at 7905.166666666666
Found 19 peaks
Fitted 2nd order to measured peaks:
 poly([ 20.75944638   0.97084012 -16.05024633])
ipykernel_launcher:12: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access

ipykernel_launcher:12: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access

ipykernel_launcher:12: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access

ipykernel_launcher:56: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

,Scan Length (um),Profile (um),Name,widths,heights,L,R
27991,4665.166667,20.7529,First Trace,136.226693,0.251584,12432.194552,13249.554712


In [38]:
for ix, row in j.split1.split2.peaks.iterrows():
    print('X=',[row['L'], row['R']])
    print('Y=', [row['heights']]*2)

X= [3408.230987654321, 3763.79]
Y= [0.062441999999999886, 0.062441999999999886]
X= [4368.684913294797, 4850.200833333333]
Y= [0.10474899999999998, 0.10474899999999998]
X= [5349.5606547619045, 5919.292]
Y= [0.14223800000000075, 0.14223800000000075]
X= [6331.233698630137, 6962.715524475524]
Y= [0.17523599999999995, 0.17523599999999995]
X= [7324.264585152839, 8039.130029673591]
Y= [0.20511800000000058, 0.20511800000000058]
X= [8346.79719851577, 9102.937034313725]
Y= [0.2242689999999996, 0.2242689999999996]
X= [9371.859284253578, 10160.848129175947]
Y= [0.2348189999999981, 0.2348189999999981]
X= [10399.447670364501, 11203.415412026727]
Y= [0.25094800000000106, 0.25094800000000106]
X= [11420.260158227848, 12226.976610169491]
Y= [0.2579419999999999, 0.2579419999999999]
X= [12432.194551724138, 13249.554712153518]
Y= [0.25158400000000114, 0.25158400000000114]
X= [13453.21442857143, 14272.458721461187]
Y= [0.25690799999999925, 0.25690799999999925]
X= [14486.710480961923, 15292.515876993166]
Y= 

In [73]:
j.split1.split2.peaks.query(f'`Profile (um)` == `Profile (um)`.max()')


,Scan Length (um),Profile (um),Name,widths,heights,L,R
27991,4665.166667,20.7529,First Trace,136.226693,0.251584,12432.194552,13249.554712
